##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [1]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from functools import partial
from multiprocessing import Pool
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
import MiscFuncs as MF
importlib.reload(Dense)
importlib.reload(RSTF)
importlib.reload(MF)

%matplotlib inline
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from matplotlib import cm

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

from ipywidgets import FloatProgress
from IPython.display import display
import os  
try:pool.close()
except:pass

Using TensorFlow backend.


In [3]:

# def ModSelect(Scope,Site):
#     if Site == 'Illisarvik':
# #         BaseFactors = ['Sedge','Shrub','Grass','Sparse','Out_of_Basin']
# #         BaseFactors = []
#         if Scope == 'Full':
#             Model = ['H','wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
#                     'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
#         if Scope == 'Test':
#             Model = ['PPFD_Avg','wind_speed']
#     if Site == 'FishIsland':
#         BaseFactors = []
#         if Scope == 'Full':
#             Model = ['H','Wind Spd','air pressure','Ta','Rn','PPFD','Rain','Water Table',
#             'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain','Wtr Tbl Trnd']
#         if Scope == 'Test':
#             Model = ['H','Water Table','Wind Spd','Active Layer']
#     return(Model)

Model = ['PPFD_Avg','wind_speed']
MP=True
Scope = 'Full'
cwd = os.getcwd()
# for Site in ['Illisarvik','FishIsland']:
Site='Illisarvik'
FillVar = 'fco2'
#     for FillVar in ['fco2','fch4']:
Runs,params = MF.Params(Scope,FillVar,MP)
params['Dpath'] = cwd+'/'+Site+'/'
if __name__ == '__main__':
    if params['proc']>1:
        pool = Pool(processes=3,maxtasksperchild=75)
    else:
        pool = None
    
    RST = RSTF.ReadStandardTimeFill(params['Dpath']+'ECData.csv')
    RST.Scale(params['Y'],Model)
    y = RST.y*1.0
    X = RST.X*1.0
    X_fill = RST.X_fill*1.0
    
    params['N']=int(y.shape[0]/30)
    
    Dense.RunNN(params,X,y,X_fill,pool)
    
    if pool is not None:
        pool.close()
print('Done!!')

0.25
((array([[-1.55980194],
       [-1.52040482],
       [-0.97661263],
       ..., 
       [-0.89946079],
       [-1.23242962],
       [-1.25851941]], dtype=float32), array([[ 0.50841558],
       [-1.20804358],
       [ 1.0532341 ],
       [-0.53855824],
       [ 1.08598495],
       [-0.33320674],
       [ 1.28597856],
       [-0.36804369],
       [ 0.64456242],
       [ 1.02203202],
       [ 1.10780787],
       [-0.20114344],
       [ 0.94426513],
       [ 1.13458192],
       [ 0.7631883 ],
       [ 1.03005123],
       [-0.13839903],
       [ 1.16331542],
       [ 0.92104846],
       [ 1.05115867],
       [ 1.14418626],
       [-0.08733711],
       [ 1.15282249],
       [ 0.88349611],
       [ 0.63359153],
       [-1.39041603],
       [-1.62601042],
       [-0.69490844],
       [-0.52743667],
       [ 0.7834031 ],
       [-0.83525831],
       [ 1.24217772],
       [ 1.16457236],
       [ 0.09258   ],
       [-0.69650847],
       [-1.25851941],
       [ 0.44497713],
       [ 0.642161

((array([[-1.55438495],
       [-1.51611841],
       [-0.98427808],
       ..., 
       [-0.88174117],
       [-1.19336748],
       [-1.21809781]], dtype=float32), array([[  6.13286555e-01],
       [ -6.09733403e-01],
       [  1.13671887e+00],
       [  1.22083592e+00],
       [  1.28880143e+00],
       [ -1.04188764e+00],
       [ -7.40411580e-01],
       [  9.68875706e-01],
       [ -3.29111129e-01],
       [ -1.11662015e-01],
       [ -1.17786038e+00],
       [  5.82972825e-01],
       [  6.53950155e-01],
       [ -1.10619128e+00],
       [  1.21413720e+00],
       [ -1.33728385e+00],
       [ -1.16098320e+00],
       [ -5.57695746e-01],
       [ -1.12535089e-01],
       [ -8.81741166e-01],
       [  1.26674902e+00],
       [  9.88892376e-01],
       [  1.08244979e+00],
       [ -3.96338612e-01],
       [ -1.01872757e-01],
       [ -1.00152040e+00],
       [  9.33999300e-01],
       [  5.95574915e-01],
       [  9.77777183e-01],
       [  9.80672061e-01],
       [ -3.96821141e-01],

((array([[-1.62106025],
       [-1.58585727],
       [-1.07744884],
       ..., 
       [-1.01627743],
       [-1.35991907],
       [-1.38777947]], dtype=float32), array([[ 0.06133694],
       [ 1.22222948],
       [-1.56375873],
       [ 0.44398528],
       [ 0.53249276],
       [ 1.20615315],
       [-0.30833596],
       [ 1.28016365],
       [ 1.18400729],
       [-0.63762957],
       [ 0.67982072],
       [ 1.26122308],
       [ 0.46322602],
       [ 1.07074273],
       [ 0.6436227 ],
       [-0.82389849],
       [ 1.26706791],
       [-0.66541666],
       [ 0.00677647],
       [-1.3111937 ],
       [ 0.56872815],
       [-1.42248201],
       [ 1.27194989],
       [ 0.84723336],
       [-0.62135106],
       [-0.42359629],
       [-0.39276016],
       [-0.81455946],
       [-0.01882857],
       [-1.06412816],
       [ 0.08104838],
       [-1.39889812],
       [ 0.62023264],
       [-0.94520998],
       [-0.58323395],
       [ 0.204512  ],
       [ 1.28188181],
       [ 1.1606456 ],


((array([[-1.59405494],
       [-1.56447029],
       [-1.10629463],
       ..., 
       [-1.06881571],
       [-1.38062143],
       [-1.40745139]], dtype=float32), array([[ 0.99461275],
       [ 0.62468636],
       [ 0.55116743],
       [ 0.94266993],
       [-0.04020307],
       [-0.51579624],
       [-0.44863829],
       [ 0.61117655],
       [-0.84973037],
       [ 0.99610382],
       [-0.32083338],
       [-0.45134938],
       [ 0.061405  ],
       [-1.08841181],
       [ 0.36383939],
       [-0.81093365],
       [ 0.35296032],
       [ 0.51397866],
       [ 1.15556049],
       [-0.08314078],
       [-0.95106858],
       [-0.56955564],
       [ 1.15686226],
       [-0.28823987],
       [-0.63324136],
       [-0.55640996],
       [-0.18862927],
       [ 0.95628721],
       [ 1.0123713 ],
       [-0.01991746],
       [-1.43471551],
       [ 0.63310599],
       [-1.32591581],
       [ 0.40978867],
       [-0.85763633],
       [ 0.86342674],
       [ 1.10636926],
       [-0.98307651],


((array([[-1.58611023],
       [-1.54982686],
       [-1.08044469],
       ..., 
       [-0.96258515],
       [-1.33702791],
       [-1.36622453]], dtype=float32), array([[ 0.5437873 ],
       [-1.26820469],
       [ 1.03563237],
       [-0.52862793],
       [ 1.08118367],
       [-0.30874774],
       [ 1.26407826],
       [-0.36576647],
       [ 0.62292701],
       [ 1.01653767],
       [ 1.10048604],
       [-0.17560118],
       [ 0.95287651],
       [ 1.15656841],
       [ 0.73262042],
       [ 1.03505242],
       [-0.19985764],
       [ 1.16355407],
       [ 0.92977339],
       [ 1.03170514],
       [ 1.15175676],
       [-0.05156637],
       [ 1.1662035 ],
       [ 0.87252432],
       [ 0.57329994],
       [-1.40922785],
       [-1.61812127],
       [-0.73255211],
       [-0.47615933],
       [ 0.76167315],
       [-0.90632069],
       [ 1.28148735],
       [ 1.16564643],
       [ 0.03824581],
       [-0.69802314],
       [-1.36622453],
       [ 0.39926517],
       [ 0.60507292],


((array([[-1.57820892],
       [-1.5427537 ],
       [-1.09475052],
       ..., 
       [-0.93622744],
       [-1.29932284],
       [-1.32766104]], dtype=float32), array([[ 0.55636579],
       [-0.56933379],
       [ 1.15594649],
       [ 1.23213005],
       [ 1.20559371],
       [-1.02887201],
       [-0.7130543 ],
       [ 0.94555902],
       [-0.29931518],
       [-0.12902805],
       [-1.16133082],
       [ 0.55312586],
       [ 0.63284808],
       [-1.16807532],
       [ 1.21341825],
       [-1.32777429],
       [-1.29480803],
       [-0.54914892],
       [-0.14498347],
       [-0.93622744],
       [ 1.2830503 ],
       [ 0.92297256],
       [ 1.05168426],
       [-0.38063982],
       [-0.11489892],
       [-0.98711509],
       [ 0.91274679],
       [ 0.56548899],
       [ 0.94827938],
       [ 0.95417225],
       [-0.37784824],
       [-1.26492977],
       [-0.02199423],
       [ 1.24619794],
       [ 0.08339792],
       [ 1.18568611],
       [ 1.2386421 ],
       [-0.66643625],


((array([[-1.60683203],
       [-1.57434428],
       [-1.06568921],
       ..., 
       [-1.04287601],
       [-1.32943237],
       [-1.35427213]], dtype=float32), array([[ 0.12824136],
       [ 1.21525657],
       [-1.53091776],
       [ 0.43337104],
       [ 0.52182889],
       [ 1.21903908],
       [-0.26819906],
       [ 1.26219606],
       [ 1.19339681],
       [-0.81291389],
       [ 0.66949451],
       [ 1.21822798],
       [ 0.48767552],
       [ 1.04568183],
       [ 0.66292107],
       [-0.78181183],
       [ 1.25328839],
       [-0.59706652],
       [-0.00557002],
       [-1.30536056],
       [ 0.55860209],
       [-1.38665974],
       [ 1.25386786],
       [ 0.8366859 ],
       [-0.78574038],
       [-0.41649747],
       [-0.39591107],
       [-0.79237509],
       [ 0.0552842 ],
       [-1.03362572],
       [ 0.09114967],
       [-1.38952661],
       [ 0.63757133],
       [-0.95128977],
       [-0.58844554],
       [ 0.1891939 ],
       [ 1.26803648],
       [ 1.17806673],


((array([[-1.66700399],
       [-1.6270262 ],
       [-1.06153929],
       ..., 
       [-0.99573797],
       [-1.29692113],
       [-1.32238925]], dtype=float32), array([[ 0.95040846],
       [ 0.68296969],
       [ 0.5527119 ],
       [ 0.90567982],
       [-0.01327746],
       [-0.49426901],
       [-0.3828488 ],
       [ 0.61037624],
       [-0.81714034],
       [ 0.9898777 ],
       [-0.23837294],
       [-0.41518217],
       [ 0.04130584],
       [-1.02031076],
       [ 0.44550556],
       [-0.88065779],
       [ 0.44286877],
       [ 0.58228129],
       [ 1.17980266],
       [-0.05787954],
       [-0.92457348],
       [-0.51228696],
       [ 1.13842988],
       [-0.24822997],
       [-0.58262116],
       [-0.53786969],
       [-0.08550814],
       [ 0.93356919],
       [ 0.95902252],
       [ 0.06089346],
       [-1.40307868],
       [ 0.61041218],
       [-1.39964283],
       [ 0.3792184 ],
       [-0.92442983],
       [ 0.80787092],
       [ 1.09117544],
       [-1.05526388],


((array([[-1.60285735],
       [-1.56499791],
       [-1.03477371],
       ..., 
       [-0.96399504],
       [-1.29682207],
       [-1.32451332]], dtype=float32), array([[ 0.54515392],
       [-1.24237895],
       [ 1.01743042],
       [-0.53607333],
       [ 1.10451293],
       [-0.30664456],
       [ 1.24939024],
       [-0.35316864],
       [ 0.6553759 ],
       [ 1.06461823],
       [ 1.07786894],
       [-0.13989031],
       [ 0.96923512],
       [ 1.16742587],
       [ 0.77443963],
       [ 1.0044266 ],
       [-0.18043208],
       [ 1.17087543],
       [ 0.95076472],
       [ 1.0107646 ],
       [ 1.10949743],
       [-0.01598099],
       [ 1.194368  ],
       [ 0.86358261],
       [ 0.62779629],
       [-1.3967036 ],
       [-1.63747466],
       [-0.74663711],
       [-0.48777008],
       [ 0.7988739 ],
       [-0.87400025],
       [ 1.26651692],
       [ 1.17428803],
       [ 0.05576355],
       [-0.68986815],
       [-1.32451332],
       [ 0.40977854],
       [ 0.6105907 ],


((array([[-1.60811388],
       [-1.57150388],
       [-1.05878019],
       ..., 
       [-0.96154314],
       [-1.28866673],
       [-1.31558347]], dtype=float32), array([[ 0.59724355],
       [-0.5923354 ],
       [ 1.16622066],
       [ 1.26271248],
       [ 1.19645917],
       [-1.02743661],
       [-0.72052532],
       [ 0.9271121 ],
       [-0.29882941],
       [-0.09320892],
       [-1.15924859],
       [ 0.54445857],
       [ 0.67612714],
       [-1.15962279],
       [ 1.24689114],
       [-1.34344029],
       [-1.26013446],
       [-0.55022764],
       [-0.11812116],
       [-0.96154314],
       [ 1.27745616],
       [ 0.92349672],
       [ 1.03252423],
       [-0.31310204],
       [-0.11273538],
       [-0.99022728],
       [ 0.93832952],
       [ 0.55909675],
       [ 0.93414396],
       [ 0.93880773],
       [-0.40272784],
       [-1.26636887],
       [-0.01238617],
       [ 1.27200007],
       [ 0.09151629],
       [ 1.15532398],
       [ 1.20788467],
       [-0.66573632],


In [ ]:
def Curve(x,a,b,c):
    return (a*x**2+b*x+c)

def TTV_Split(iteration,Memory,X,y,params,X_fill):
    params['seed'] = int(iteration%params['splits_per_mod']/params['splits_per_mod']*100)
    params['iteration'] = int(iteration/params['splits_per_mod'])
    X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.1, random_state=params['seed'])
    X_train,X_val,y_train,y_val=train_test_split(X_train,y_train, test_size=0.11, random_state=params['seed'])
    return(Dense.Train_Steps(params,X_train,X_test,X_val,y_train,y_test,
        y_val,X_fill = X_fill,Memory=Memory),
        y_val)

def RunReps(Model,params,pool = None,FullReturn=False):
    RST = RSTF.ReadStandardTimeFill(params['Dpath']+'ECData.csv')
    offset = 5/params['proc']
    Memory = (math.floor(100/params['proc'])- offset) * .01
    MSE = []
    RST.Scale(params['Y'],Model)
    y = RST.y*1.0
    X = RST.X*1.0
    X_fill = RST.X_fill*1.0
    MAE = []
    R2 =[]
    Y_fill = []
    if params['Prelim_N']==True:
        params['N']=int(y.shape[0]*.8/30)
    if __name__=='__main__'and params['proc'] != 1:
        for i,results in enumerate(pool.imap(partial(TTV_Split,Memory=Memory,X=X,y=y,params=params,X_fill=X_fill),range(params['K']))):
            Y_fill.append(results[0][0])
            Yval = RST.YScaled.inverse_transform(results[0][1].reshape(-1,1))
            y_val = RST.YScaled.inverse_transform(results[1].reshape(-1,1))
            MAE.append(metrics.mean_absolute_error(Yval,y_val))
            R2.append(metrics.r2_score(Yval,y_val))
    else:
        for i in range(params['K']):
            results = TTV_Split(i,Memory,X,y,params,X_fill)
            Y_fill.append(results[0][0])
            Yval = RST.YScaled.inverse_transform(results[0][1].reshape(-1,1))
            y_val = RST.YScaled.inverse_transform(results[1].reshape(-1,1))
            MAE.append(metrics.mean_absolute_error(y_val,Yval))
            R2.append(metrics.r2_score(y_val,Yval))
            
            
    
    tempK = params['K']+0
    params['K']=1

    params['Loss'] = 'Variance_Loss'
    y = Yfill.std(axis=0)**2
    X = X_fill

    results = TTV_Split(i,Memory,X,y,params,X_fill)
    params['K'] = tempK
    Y_var = (RST.YScaled.inverse_transform(results[0][0].reshape(-1,1))).mean(axis=-1)
#         Yval=RST.YScaled.inverse_transform(results[0][1].reshape(-1,1))
#         y_val=RST.YScaled.inverse_transform(results[1].reshape(-1,1))
#         print(Y_var.shape)
    MSE_Boot = ((Yfill-Yfill_bar)**2).sum(axis=0)
    SE = stats.t.ppf(1-0.025,params['K'])*(1/(params['K']-1)*(MSE_Boot))**.5
    PI = 1.96*Y_var**.5
    print(SE.shape)
    RST.Master['CI'] = SE
    RST.Master['PI'] = Y_var
    plt.figure()
    plt.bar(RST.Master.index,RST.Master['TempFill'],RST.Master['PI'])
            
    Y_fill = np.asanyarray(Y_fill)
    Y_fill = Y_fill.mean(axis=0) 
    FillVarName = FillVar.replace('f','F')
    RST.Fill(Y_fill,FillVarName)
    return(MAE,R2,RST.Master['TempFill'])

def RunLoop(Runs,model):
    style = {'description_width': 'initial'}
    f = FloatProgress(min=0, max=Runs['N'].count(),description='Finding Best N:',style=style) # instantiate the bar
    display(f) # display the bar
    i = 0
    for N in Runs['N'].unique():
        params['T'] = 0
        params['N']=N
        Results = RunReps(model,params,pool)
        MAE = Results[0]
        R2 = Results[1]
        Runs.loc[Runs['N']==N,'MAE']=MAE
        Runs.loc[Runs['N']==N,'R2']=R2
        f.value+=params['K']
        MAE = Runs.loc[Runs['MAE']!=0].groupby('N').mean()#.values[0]
        MAE['se'] = Runs.loc[Runs['MAE']!=0,['N','MAE']].groupby('N').sem()#.values[0]
        Val = MAE['MAE'].min()+MAE.loc[MAE['MAE']==MAE['MAE'].min(),'se'].values[0]
        if i > 2:
            if np.asanyarray(MAE).mean()>Val:
                Runs = Runs.loc[Runs['MAE']!=0]
                print('Leaving at ',N)
                break
        i+=1                
    return(Runs)

def N_Nodes(Runs,params,model,plot=False):
    params['Prelim_N']=False
    Runs = RunLoop(Runs,model)
    Group = Runs.groupby('N').mean()
    Group['Count'] = Runs[['N','MAE']].groupby('N').count()
    Group['SD'] = Runs[['N','MAE']].groupby('N').std()
    Group['SE'] = Runs[['N','MAE']].groupby('N').sem()
    Group = Group.reset_index()
    Min = Group.loc[Group['MAE']==Group['MAE'].min()]
    popt, pcov = curve_fit(Curve, Runs['N'].values, Runs['MAE'].values)
    N = np.arange(Runs['N'].min(),Runs['N'].max())
    Fit = Curve(N,*popt)
    Max = (Min['MAE']+Min['SE']).values[0]
    try:
        params['N']=N[Fit<Max].min()
    except:
        print('Fitting failed to produce meaningful results')
        params['N'] = Group.loc[Group['MAE']<+Min['MAE'].values[0]+Min['SE'].values[0],'N'].values[0]
        pass
    
    if plot == True:
        fig,ax = plt.subplots(figsize=(8,8))
        ax.scatter(Runs['N'],Runs['MAE'])
        ax.plot(N,Fit,color='r')
        ax.plot(N,N*0+Max,color = 'black')
        plt.grid()
        plt.show()
    return(params,Runs,Group)

def FactorTest(params,FullModel,Runs):
    prog1 = FloatProgress(min=0, max=len(FullModel)-1,description='FactorTesting:') # instantiate the bar
    display(prog1) # display the bar
    Scores = pd.DataFrame()
    ModelRuns = pd.DataFrame()
    Start = 1
    for level in range(Start,len(FullModel)+1):
        if level == 1:
            Models = MF.Combos(FullModel,level)
        else:
            Models = MF.Combos(FullModel,level,factor = Best)
        print('Number of Models: ',len(Models))
        prog2 = FloatProgress(min=0, max=len(Models),description='Level: '+str(level)) # instantiate the bar
        display(prog2) # display the bar
        Values = []
        for j,model in enumerate(Models):
            key = str(level)+'_'+str(j)
            Name = '+'.join(model)
            Results = RunReps(model,params,pool)
            MAE = Results[0]
            R2 = Results[1]
            d = {'Model':'+'.join(model),'MAE':MAE,'Key':key,'R2':R2,'Level':level}#,'Best':'N'
            score = pd.DataFrame(index=[x for x in range(params['K'])],data=d)
            Scores = Scores.append(score,ignore_index=True)## Might cause problems in older versions
            ModelRuns['Model: '+'+'.join(model)] = Results[2]    
            prog2.value+=1
        Group = Scores.groupby('Key').agg({'Model': 'first','Level': 'first','MAE': 'mean','R2': 'mean'}) 
        Group['SE'] = Scores[['Key','MAE']].groupby('Key').sem()
        Min = Group.loc[Group['Level']==level]
#         print(Min)
        Mod2Beat = Min.loc[Min['MAE'] == Min['MAE'].min()]
        print('Best Model @ Level ',level)
        print(Mod2Beat)
        if level>Start:
            print('Should we exit??', (Mod2Beat['MAE']+Mod2Beat['SE']).values[0],LastMin)
            if (Mod2Beat['MAE']+Mod2Beat['SE']).values[0]>LastMin:
                print('Exit Early')
                Scores = Scores.loc[Scores['Level']<level]
                break
        LastMin = Min['MAE'].min()
        Best = Mod2Beat['Model'].values
        prog1.value+=1
    Group = Scores.groupby('Key').agg({'Model': 'first','Level': 'first','MAE': 'mean','R2': 'mean'})
    Group['SE'] = Scores[['Key','MAE']].groupby('Key').sem()
    Min = Group.loc[Group['Level']==level]
    return(Best,Scores,ModelRuns)

def Best_Fill(Best,Runs,Scores,params):
    N = Runs['N']
    NRuns = Runs*0
    NRuns['N']=N
    model = Best[0].split('+')

    params,Runs,Group = N_Nodes(NRuns,params,model,plot=True)
    level = 0
    key = '0_'+str(params['N'])
    Name = '+'.join(Best)
    Results = RunReps(model,params,pool)
    MSE = Results[0]
    R2 = Results[1]
    d = {'Model':'+'.join(model),'MAE':MSE,'Key':key,'R2':R2,'Level':level}#,'Best':'N'
    score = pd.DataFrame(index=[x for x in range(params['K'])],data=d)
    Scores = Scores.append(score,ignore_index=True)
    ModelRuns['BestModel: '+'+'.join(model)] = Results[2] 
    print(params['N'])
    return(Scores,ModelRuns)

# The "Optimum" Sized Model

In [ ]:
# pool.close()
# Site = 'Illisarvik'#'FishIsland'#
Scope = 'Test'
cwd = os.getcwd()
def ModSelect(Scope,Site):
    if Site == 'Illisarvik':
#         BaseFactors = ['Sedge','Shrub','Grass','Sparse','Out_of_Basin']
#         BaseFactors = []
        if Scope == 'Full':
            Model = ['H','wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
        if Scope == 'Test':
            Model = ['PPFD_Avg','wind_speed']
    if Site == 'FishIsland':
        BaseFactors = []
        if Scope == 'Full':
            Model = ['H','Wind Spd','air pressure','Ta','Rn','PPFD','Rain','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain','Wtr Tbl Trnd']
        if Scope == 'Test':
            Model = ['H','Water Table','Wind Spd','Active Layer']
    return(Model)

# def Params(Func,Y,MP = True):
#     params = {}
#     params['proc']=3
#     if MP == False:
#         params['proc']=1
#     if Func == 'Full':
#         epochs = 200
#         K = 30
#         splits_per_mod = 1
#         N = np.linspace(200,20,10,dtype='int32')
#     elif Func == 'Test':
#         epochs = 200
#         K = 30
#         splits_per_mod = 1
#         N = np.linspace(70,10,5,dtype='int32')
#     N = np.repeat(N,K)
#     d = {'N':N.astype(int)}
#     Runs = pd.DataFrame(data=d)
#     Runs['MAE'] = 0.0
#     Runs['R2'] = 0.0
#     Runs['Model']=0
#     params['K'] = K
#     params['epochs'] = epochs
#     params['Y'] = Y
#     params['splits_per_mod'] = splits_per_mod
#     params['Save'] = {}
#     params['Save']['Weights']=False
#     params['Save']['Model']=False
    
#     return(Runs,params)


MP=False

if Scope == 'Full':
    MP = True
if __name__=='__main__'and MP==True:
    pool = Pool(processes=3,maxtasksperchild=75)
else:pool=None
    
# for Site in ['Illisarvik','FishIsland']:
Site='Illisarvik'
FillVar = 'fco2'
#     for FillVar in ['fco2','fch4']:
Runs,params = MF.Params(Scope,FillVar,MP)
FullModel = ModSelect(Scope,Site)
print(FullModel)
params['Dpath'] = cwd+'/'+Site+'/'
params['Prelim_N']=True
Best,Scores,ModelRuns = MF.FactorTest(params,FullModel,Runs)
# print(Best,Scores)
# Scores,ModelRuns = Best_Fill(Best,Runs,Scores,params)
# Scores.to_csv(params['Dpath']+FillVar+'/GapFillingSummary.csv')
# ModelRuns.to_csv(params['Dpath']+FillVar+'/GapFilled.csv')

# if __name__=='__main__'and MP==True:
#     pool.close()

In [ ]:
# Grp = Scores.groupby('Model').mean()
# Grp['SE'] = Scores[['Model','MAE']].groupby('Model').sem()
# # Grp['SE'] = Scores[['Key','MAE']].groupby('Key').sem()
# print(Grp)
# # plt.bar(Grp.index,Grp['MAE'],yerr=Grp['SE'])

In [ ]:
print('kitty')